In [5]:
%pip install llama-index-llms-ollama
%pip install llama-index-program-lmformatenforcer
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.73-cp312-cp312-macosx_14_0_arm64.whl size=3469930 sha256=9210c3aa63e64b5e0f0ce3680d440fe682022398fb145b0e649acbde227e1443
  Stored in directory: /Users/greg/Library/Caches/pip/wheels/44/8f/83/bed7814c35c6945f720c80bd458bf0f4a053cd551221e4e6a6
Successfully built llama-cpp-python
  Attempting uninstall: huggingface-hub
    Found existing installation: h

In [6]:
%pip install llama-index lm-format-enforcer llama-cpp-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.2 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys

from pydantic import BaseModel, Field
from typing import List

from llama_index.program.lmformatenforcer import (
    LMFormatEnforcerPydanticProgram,
)

/Users/greg/.pyenv/versions/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/greg/.pyenv/versions/3.12.1/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [348]:
from pydantic import BaseModel, validator
from typing import Union
from datetime import date, datetime
from typing import Optional
import json

class DateEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, date):
            return obj.isoformat()
        return super().default(obj)

class Address(BaseModel):
    street: Optional[str] = None
    city: Optional[str] = None
    state: Optional[str] = None
    postal_code: Optional[str] = None
    country: Optional[str] = None

    def json(self, *args, **kwargs):
        return json.dumps(self.dict(), cls=DateEncoder)

class Donor(BaseModel):
    name: Optional[str] = None
    address: Optional[Address] = None

    def json(self, *args, **kwargs):
        return json.dumps(self.dict(), cls=DateEncoder)

class Money(BaseModel):
    value: Optional[float] = None
    currency: Optional[str] = None

    def json(self, *args, **kwargs):
        return json.dumps(self.dict(), cls=DateEncoder)

class Donation(BaseModel):
    donor: Optional[Donor] = None
    amount: Optional[Money] = None
    date: Optional[datetime] = None
    description: Optional[str] = None

    def json(self, *args, **kwargs):
        return json.dumps(self.dict(), cls=DateEncoder)

class Interest(BaseModel):
    description: Optional[str] = None

    def json(self, *args, **kwargs):
        return json.dumps(self.dict(), cls=DateEncoder)

class Item(BaseModel):
    donation: Optional[Union[Donation, None]] = None
    interest: Optional[Union[Interest, None]] = None
    registered_date: Optional[date] = None

    @validator('registered_date', pre=True, always=True)
    def parse_date(cls, value):
        if value is None:
            return None
        try:
            return datetime.strptime(value, '%Y-%m-%d').date()
        except (ValueError, TypeError):
            return None

    @validator('donation', 'interest', pre=True)
    def check_exclusive(cls, v, values):
        if v is not None and values.get('interest') is not None:
            raise ValueError("An item can only be either a donation or an interest, not both.")
        return v

    def json(self, *args, **kwargs):
        return json.dumps(self.dict(), cls=DateEncoder)


/var/folders/1m/xcsrw32d7mnbpc723_dd8_f00000gp/T/ipykernel_53456/3725947808.py:57: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('registered_date', pre=True, always=True)
/var/folders/1m/xcsrw32d7mnbpc723_dd8_f00000gp/T/ipykernel_53456/3725947808.py:66: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('donation', 'interest', pre=True)


In [349]:
from llama_index.llms.llama_cpp import LlamaCPP

llm = LlamaCPP(temperature=0.0)

ITEM = """"Name of donor: Harlequin Football Club Ltd
Address of donor: Twickenham Stoop Stadium, Langhorn Drive, Twickenham TW2 7SX
Amount of donation or nature and value if donation in kind: Two tickets with hospitality for the BG14 match at Twickenham Stadium, value Â£900
Date received: 4 March 2023
Date accepted: 4 March 2023
Donor status: company, registration 03213073
(Registered 06 March 2023)"""

program = LMFormatEnforcerPydanticProgram(
    output_cls=Item,
    prompt_template_str=(
        "Your response should be according to the following json schema: \n"
        "{json_schema}\n"
        """You are AI assistant that extracts entities from the free text descriptions of MP's items.

                   The items are either donations or interests. If there is no financial value, then assume it is an interest.

                   Ensure to record a description of the donation or interest.

                   Ensure you extract a financial value of donation where available.
                   
                   If any parts of the template don't look relevant, leave them blank.
                
                   Try and record the dates items are registered, usually you can find this by looking for the word "Registered" followed by the date.

                   Ensure dates are always in ISO format.
                   
                   Do NOT make up information if you are unsure.

                   Output nothing other than valid JSON.

                   """
        "This is the item to analyse:\n\n"
        "{item}"
    ),
    llm=llm,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/greg/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

In [350]:
expense_description = "Name of donor: Ministry of Foreign Affairs of the State of Qatar\nAddress of donor: Ministry of Foreign Affairs, Almirqab Tower, West Bay, Doha, Qatar\nEstimate of the probable value (or amount of any donation): Return flights, accommodation and food, estimated value \u00c2\u00a36,872\nDestination of visit: Doha, Qatar\nDates of visit: 19-23 September 2023\nPurpose of visit: Qatar APPG visit to meet with ministers, the Shura Council and senior local and international stakeholders in international development to explore ways of enhancing UK-Qatar cooperation on global development and aid.\n(Registered 16 October 2023)"

output = program(item=expense_description)

ERROR:root:Unknown LMFormatEnforcer Problem. Prefix: 'Your response should be according to the following json schema: 
{"$defs": {"Address": {"properties": {"street": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Street"}, "city": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "City"}, "state": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "State"}, "postal_code": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Postal Code"}, "country": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Country"}}, "title": "Address", "type": "object"}, "Donation": {"properties": {"donor": {"anyOf": [{"$ref": "#/$defs/Donor"}, {"type": "null"}], "default": null}, "amount": {"allOf": [{"$ref": "#/$defs/Money"}], "default": null}, "date": {"anyOf": [{"format": "date-time", "type": "string"}, {"type": "null"}], "default": null, "title": "Date"}, "description": {"

ValidationError: 1 validation error for Item
__root__
  Unterminated string starting at: line 15 column 1 (char 28) [type=value_error.jsondecode, input_value='\n\n\n\n\n\n\n\n\n\n\n\n...n"donation": {\n"amount', input_type=str]

In [345]:
output


Item(donation=Donation(donor=Donor(name='Ministry of Foreign Affairs of the State of Qatar', address=Address(street='Ministry of Foreign Affairs, Almirqab Tower, West Bay', city='Doha', state='Qatar', postal_code='Qatar', country='Qatar')), amount=None, date=None, description='Return flights, accommodation and food, estimated value £6,872'), interest=None, registered_date=datetime.date(2023, 10, 16))

In [339]:



test = output.json(indent=None, separators=(',', ':'))
print(test)
with open(file_path, 'a') as file:
    file.write(f"{test}\n")

{"donation": null, "interest": {"description": "Member of the Teesside Freeport Board"}, "registered_date": null}


In [340]:
with open('items.json') as f:
    data = json.load(f)

# Assuming the JSON structure is like {"array_key": [item1, item2, ...]}
# Access the array within the JSON data
array_items = data['rows']

# Path to the JSON lines file
file_path = "output.jsonl"

# Open the file in write mode
with open(file_path, 'a') as file:
    # Iterate over the items in the array
    for index, item in enumerate(array_items):
        expense_description = item[1]
        # print(f"Input: {expense_description}\n")
        parsed_description = program(item=expense_description).json(indent=None, separators=(',', ':'))
        file.write(f"{parsed_description}\n")
        # print(json_str)
        # Add a newline after each JSON object
        file.flush()
        print(index)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7677.86 ms
llama_print_timings:      sample time =       8.68 ms /    85 runs   (    0.10 ms per token,  9788.12 tokens per second)
llama_print_timings: prompt eval time =  470178.72 ms /   168 tokens ( 2798.68 ms per token,     0.36 tokens per second)
llama_print_timings:        eval time =    7809.98 ms /    84 runs   (   92.98 ms per token,    10.76 tokens per second)
llama_print_timings:       total time =    8052.84 ms /   252 tokens
Llama.generate: prefix-match hit


0



llama_print_timings:        load time =    7677.86 ms
llama_print_timings:      sample time =       7.63 ms /    77 runs   (    0.10 ms per token, 10093.07 tokens per second)
llama_print_timings: prompt eval time =    2129.24 ms /    42 tokens (   50.70 ms per token,    19.73 tokens per second)
llama_print_timings:        eval time =    6508.50 ms /    76 runs   (   85.64 ms per token,    11.68 tokens per second)
llama_print_timings:       total time =    8773.54 ms /   118 tokens
Llama.generate: prefix-match hit


1



llama_print_timings:        load time =    7677.86 ms
llama_print_timings:      sample time =       8.12 ms /    80 runs   (    0.10 ms per token,  9851.00 tokens per second)
llama_print_timings: prompt eval time =    2094.18 ms /    34 tokens (   61.59 ms per token,    16.24 tokens per second)
llama_print_timings:        eval time =    6755.71 ms /    79 runs   (   85.52 ms per token,    11.69 tokens per second)
llama_print_timings:       total time =    8994.24 ms /   113 tokens
Llama.generate: prefix-match hit


...

33



llama_print_timings:        load time =    7677.86 ms
llama_print_timings:      sample time =      17.63 ms /   185 runs   (    0.10 ms per token, 10492.88 tokens per second)
llama_print_timings: prompt eval time =    2168.37 ms /    50 tokens (   43.37 ms per token,    23.06 tokens per second)
llama_print_timings:        eval time =   15965.10 ms /   184 runs   (   86.77 ms per token,    11.53 tokens per second)
llama_print_timings:       total time =   18481.74 ms /   234 tokens
Llama.generate: prefix-match hit


34



llama_print_timings:        load time =    7677.86 ms
llama_print_timings:      sample time =      23.27 ms /   256 runs   (    0.09 ms per token, 10999.40 tokens per second)
llama_print_timings: prompt eval time =    3787.24 ms /   177 tokens (   21.40 ms per token,    46.74 tokens per second)
llama_print_timings:        eval time =   22520.34 ms /   255 runs   (   88.32 ms per token,    11.32 tokens per second)
llama_print_timings:       total time =   26854.29 ms /   432 tokens


ValidationError: 1 validation error for Item
__root__
  Unterminated string starting at: line 23 column 1 (char 555) [type=value_error.jsondecode, input_value='\n\n\n\n\n\n\n\n\n\n\n\n...tes Ltd"\n}\n},\n"inter', input_type=str]

In [313]:
parsed_description

Item(donation=Donation(donor=None, amount=Money(value=100.0, currency='CAD'), date=None, description='A donation of $100 to support the local animal shelter'), interest=None, registered_date=None)

In [317]:
type(json.dumps((json_str)))

str